# Modelo final

In [ ]:
cliente_id = 932

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

# Cargar y preparar datos
ordenes = pd.read_csv('ordenes.csv')
socios = pd.read_csv('socios_proveedores.csv')
platillos = pd.read_csv('platillos.csv')
diccionario=pd.read_csv('diccionario.csv')
def limpiar(texto):
    return re.sub(r'[^a-záéíóúñü\s]', '', str(texto).lower())

# Perfiles del cliente (por categoría)
ordenes = ordenes.merge(socios[['NumeroProveedor', 'CategoryId']], on='NumeroProveedor', how='left')
perfil_cliente = ordenes.groupby(['NumeroSocioConsumidor', 'CategoryId']).size().unstack(fill_value=0)
perfil_cliente = perfil_cliente.div(perfil_cliente.sum(axis=1), axis=0)

# Perfiles de restaurantes
platillos = platillos.dropna(subset=['Description'])
platillos['clean_desc'] = platillos['Description'].apply(limpiar)
desc_rest = platillos.groupby('NumeroProveedor')['clean_desc'].apply(' '.join).reset_index()

tfidf_vec = TfidfVectorizer(max_features=500)
rest_tfidf = tfidf_vec.fit_transform(desc_rest['clean_desc'])

cat_matrix = (
    socios[['NumeroProveedor', 'CategoryId']]
    .assign(value=1)
    .pivot_table(index='NumeroProveedor', columns='CategoryId', values='value', fill_value=0)
    .loc[desc_rest['NumeroProveedor']]
)

rest_vecs = hstack([rest_tfidf, cat_matrix.values])

#Alinear columnas cliente-restaurante 
for col in cat_matrix.columns.difference(perfil_cliente.columns):
    perfil_cliente[col] = 0
perfil_cliente = perfil_cliente[cat_matrix.columns]

# Vector del cliente (TF-IDF + perfil categoría) 
ordenes['clean_desc'] = ordenes['Description'].fillna('').apply(limpiar)
desc_cliente = ordenes.groupby('NumeroSocioConsumidor')['clean_desc'].apply(' '.join).reset_index()

# Reconstruir vectores de clientes
desc_cliente = desc_cliente[desc_cliente['NumeroSocioConsumidor'].isin(perfil_cliente.index)].reset_index(drop=True)
cli_idx_map = dict(zip(desc_cliente['NumeroSocioConsumidor'], desc_cliente.index))
tfidf_cli = tfidf_vec.transform(desc_cliente['clean_desc'])
cli_vecs = hstack([tfidf_cli, perfil_cliente.loc[desc_cliente['NumeroSocioConsumidor']].values]).tocsr()



In [ ]:

import pandas as pd

# Cargar la tabla de órdenes
ordenes = pd.read_csv('ordenes.csv')

# Contar cuántas órdenes tiene cada cliente
top_clientes = (
    ordenes['NumeroSocioConsumidor']
        .value_counts()          # cuenta apariciones
        .reset_index()           # vuelve DataFrame
        .rename(columns={'index': 'NumeroSocioConsumidor', 
                         'NumeroSocioConsumidor': 'total_ordenes'})
)

top = (ordenes
       .groupby('NumeroSocioConsumidor')
       .size()
       .sort_values(ascending=False)
       .head(10))

top


NumeroSocioConsumidor
49       121
28686    107
932      106
28639     98
27        87
34132     76
45298     75
65        67
28097     67
577       67
dtype: int64

In [ ]:
import pandas as pd

# renombramos el diccionario una sola vez
dic = (
    diccionario.rename(columns={
        'Category Id':    'CategoryId',
        'Subcategory Id': 'SubcategoryId',
        'Categoría':      'Categoria',
        'Subcategoría':   'Subcategoria'
    })
)


def recomendar(cliente_id, n=5):
    #Checamos que exista
    if cliente_id not in cli_idx_map:
        raise ValueError("Cliente no encontrado 🤔")
    
    # Similitud coseno cliente-vs-restaurantes
    idx  = cli_idx_map[cliente_id]
    sims = cosine_similarity(cli_vecs[idx], rest_vecs).flatten()

    # DataFrame base
    resultados = desc_rest[['NumeroProveedor']].copy()
    resultados['similarity'] = sims

    # Añadimos datos del restaurante (incluye ambos IDs)
    resultados = resultados.merge(
        socios[['NumeroProveedor', 'CategoryId', 'SubcategoryId',
                'NombreRestaurante', 'latitud', 'longitud']],
        on='NumeroProveedor', how='left'
    )

    # Añadimos nombres de categoría y subcategoría 😎
    resultados = resultados.merge(
        dic[['CategoryId', 'Categoria']], on='CategoryId', how='left'
    ).merge(
        dic[['SubcategoryId', 'Subcategoria']], on='SubcategoryId', how='left'
    )

    # Orden + diversidad (top-2 por categoría)
    resultados = (resultados
                  .sort_values(by='similarity', ascending=False)
                  .groupby('CategoryId', group_keys=False)
                  .head(1))
    
    # Regresamos sólo lo que quieres mostrar
    cols = ['NombreRestaurante', 'Categoria', 'Subcategoria',
            'similarity', 'latitud', 'longitud']
    return resultados[cols].head(n)

recs = recomendar(cliente_id, 5)
recs


,NombreRestaurante,Categoria,Subcategoria,similarity,latitud,longitud
276,La Casa del Panque,Cafetería,Cafeterias,0.701069,25.670233,-100.304648
677,Reina Comfort Food+Cafe,Desayunos y brunch,Brunch,0.599111,25.654232,-100.362532
988,John Ham's Fashion Drive,Americana,Hamburguesas,0.574318,25.651110,-100.334829
236,El Semillerito Restaurante,Italiana,Pastas,0.569837,25.672227,-100.325418
998,John Ham's Galerias Monterrey,Rápida,Hamburguesas,0.486343,25.680381,-100.356458


In [ ]:
import folium
# ID del cliente y coords fijas
USER_LAT, USER_LON = 25.659990, -100.369792   # 👈 zona Tec exacta
# ----------------------------------

if recs.empty:
    print(f"No hay recs para el usuario {cliente_id}")
else:
    # Mapa centrado en la ubicación del user
    m = folium.Map(location=[USER_LAT, USER_LON], zoom_start=14)

    # Punto rojo = tú
    folium.Marker(
        [USER_LAT, USER_LON],
        tooltip="Tú estás aquí",
        icon=folium.Icon(color="red", icon="home")
    ).add_to(m)

    # Marcadores de restaurantes recomendados
    for _, r in recs.iterrows():
        popup_txt = (f"<b>{r.NombreRestaurante}</b><br>"
                     f"Similitud: {r.similarity*100:.1f}%<br>"
                     f"{r.Categoria} / {r.Subcategoria}")
        folium.Marker(
            [r.latitud, r.longitud],
            popup=popup_txt,
            tooltip=f"{r.NombreRestaurante} ({r.similarity*100:.0f}%)"
        ).add_to(m)

m


In [ ]:
import pandas as pd

#  Carga de datos
ordenes      = pd.read_csv('ordenes.csv')
socios       = pd.read_csv('socios_proveedores.csv')
diccionario  = pd.read_csv('diccionario.csv')

#  Renombra las columnas largas del diccionario solo para facilitar la vida
dic = diccionario.rename(columns={
    'Category Id':    'CategoryId',
    'Subcategory Id': 'SubcategoryId',
    'Categoría':      'Categoria',      # <-- nombre bonito
    'Subcategoría':   'Subcategoria'    # <-- nombre bonito
})

#Enriquecemos socios con los nombres de categoría y subcategoría
socios_ext = (
    socios
      .merge(dic[['CategoryId', 'Categoria']],       on='CategoryId',   how='left')
      .merge(dic[['SubcategoryId', 'Subcategoria']], on='SubcategoryId', how='left')
)

# Histórico de órdenes para tu cliente
historico = ordenes[ordenes.NumeroSocioConsumidor == cliente_id]

# Merge final para traer NombreRestaurante + Categoria + Subcategoria
hist = historico.merge(
    socios_ext[['NumeroProveedor', 'NombreRestaurante', 'Categoria', 'Subcategoria']],
    on='NumeroProveedor',
    how='left'
)

#  Selecciona las columnas que quieres ver
hist = hist[['OrderID', 'NombreRestaurante', 'Categoria', 'Subcategoria',
             'MontoTotal', 'Name']]   # ajusta 'Name' si en tu CSV tiene otro título

hist

,OrderID,NombreRestaurante,Categoria,Subcategoria,MontoTotal,Name
0,48596,Tacotote,Rápida,Tacos,744.45,Agua de sabor
1,48596,Tacotote,Rápida,Tacos,744.45,Agua de sabor
2,48596,Tacotote,Rápida,Tacos,744.45,Agua de sabor
3,48596,Tacotote,Rápida,Tacos,744.45,Agua de sabor
4,48596,Tacotote,Rápida,Tacos,744.45,Agua de sabor
...,...,...,...,...,...,...
448,52735,John Ham's Valle Oriente,Americana,Hamburguesas,2355.60,Pepperoni pizza
449,52735,John Ham's Valle Oriente,Americana,Hamburguesas,2355.60,Pepperoni pizza
450,52735,John Ham's Valle Oriente,Americana,Hamburguesas,2355.60,Pepperoni pizza
451,52735,John Ham's Valle Oriente,Americana,Hamburguesas,2355.60,Pepperoni pizza


In [ ]:
import pandas as pd
import folium

ordenes = pd.read_csv('ordenes.csv')
visits = ordenes[ordenes.NumeroSocioConsumidor == cliente_id]
if visits.empty:
    print(f"No hay visitas para el usuario {cliente_id}")
else:
    
    lat_c = visits['latitud'].mean()
    lon_c = visits['longitud'].mean()

    #  mapa y  marcadores
    m = folium.Map(location=[lat_c, lon_c], zoom_start=14)
    for _, r in visits.iterrows():
        folium.Marker([r.latitud, r.longitud], popup=r.Name).add_to(m)
m


In [ ]:
import random
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# Definir el área metropolitana de Monterrey (aprox. bounding box)
LAT_MIN, LAT_MAX = 25.6134, 25.8260
LON_MIN, LON_MAX = -100.4518, -100.2137

def asignar_ubicacion_random():
    """Devuelve una tupla (lat, lon) random dentro de Monterrey."""
    lat = random.uniform(LAT_MIN, LAT_MAX)
    lon = random.uniform(LON_MIN, LON_MAX)
    return lat, lon



In [20]:

def haversine(lat1, lon1, lat2, lon2):
    """Calcula la distancia en km entre dos puntos lat/lon."""
    R = 6371  # radio tierra en km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c


In [ ]:
user_lat, user_lon=25.659990, -100.369792
print(f"Ubicación user asignada: {user_lat:.5f}, {user_lon:.5f}")



Ubicación user asignada: 25.65999, -100.36979


In [ ]:

# Calculamos la distancia de cada restaurante al user
recs['distance_km'] = recs.apply(
    lambda row: haversine(user_lat, user_lon, row['latitud'], row['longitud']),
    axis=1
)


In [ ]:
# Ordenamos primero por similarity (antes rec_score) y luego por distancia
recs_ordenadas = recs.sort_values(
    by=['distance_km','similarity'],
    ascending=[True, True]
)
top5 = (
    recs_ordenadas
        .drop_duplicates(subset='NombreRestaurante')  # o 'NombreRestaurante'
        .head(5)
)

print("Top 10 recomendaciones cercanas y afines:")
top5


Top 10 recomendaciones cercanas y afines:


,NombreRestaurante,Categoria,Subcategoria,similarity,latitud,longitud,distance_km
677,Reina Comfort Food+Cafe,Desayunos y brunch,Brunch,0.599111,25.654232,-100.362532,0.969282
998,John Ham's Galerias Monterrey,Rápida,Hamburguesas,0.486343,25.680381,-100.356458,2.631885
988,John Ham's Fashion Drive,Americana,Hamburguesas,0.574318,25.651110,-100.334829,3.640926
236,El Semillerito Restaurante,Italiana,Pastas,0.569837,25.672227,-100.325418,4.650786
276,La Casa del Panque,Cafetería,Cafeterias,0.701069,25.670233,-100.304648,6.627679


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

# Crear GeoDataFrame de recomendaciones ---
geometry = [Point(xy) for xy in zip(top5['longitud'], top5['latitud'])]
gdf_recs = gpd.GeoDataFrame(top5, geometry=geometry, crs="EPSG:4326")

#  GeoDataFrame de la ubicación del usuario ---
user_geom = Point(user_lon, user_lat)
gdf_user = gpd.GeoDataFrame(
    pd.DataFrame({'Nombre': ['Usuario']}),
    geometry=[user_geom],
    crs="EPSG:4326"
)

# Iniciar el mapa folium centrado en el usuario 
m = folium.Map(location=[user_lat, user_lon], zoom_start=13)

#  Añadir marcadores de restaurantes 
for _, row in gdf_recs.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['NombreRestaurante'],
        icon=folium.Icon(color='blue', icon='cutlery', prefix='fa')
    ).add_to(m)

#  Añadir marcador de usuario
folium.Marker(
    location=[user_lat, user_lon],
    popup='Usuario',
    icon=folium.Icon(color='red', icon='star')
).add_to(m)
m
